# List of Song Names

In [113]:
import os
import random
os.getcwd()
os.chdir("C:\\Users\\chapp\\Documents\\GitHub\\fma\\List")

In [114]:
AUDIO_DIR='C:/Users/chapp/Courses/Python/Project'

In [115]:
filenames = next(os.walk("C:\\Users\\chapp\\Documents\\GitHub\\fma\\List"))[2]

In [116]:
filenames = [x.strip('.mp3') for x in filenames]

In [117]:
os.chdir("C:\\Users\\chapp\\Documents\\GitHub\\fma")

# Clustering

In [161]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pygame
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [162]:
music = pd.read_csv('Merged.csv', header = 0, encoding = 'latin1')
col_names = ['acousticness', 'danceability', 'energy', \
                       'instrumentalness', 'liveness', 'speechiness', \
                       'tempo', 'valence']
music_numerical = music[col_names]
music_matrix = music_numerical.as_matrix()
scaler = StandardScaler()
scaler.fit(music_matrix)
new_matrix = scaler.transform(music_matrix)
music_std = pd.DataFrame(new_matrix, columns = col_names)

music_std = music_std[col_names]

In [163]:
music_std.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
0,-0.281499,0.991915,0.348732,-1.742887,-0.064267,0.437749,1.223573,0.495983
1,-0.391656,0.217484,1.006860,-1.767172,-0.518358,2.639805,0.110726,-0.617788
2,-1.253885,1.358338,0.589683,-1.770366,1.172705,0.185049,-0.651745,0.659017
3,1.112791,0.898745,1.391929,0.898939,-0.457654,-0.481812,-0.328958,1.897809
4,-0.188870,0.136465,0.082343,-1.718498,-0.577286,3.103507,-0.251046,1.645950


# Neural Network For Missing Genres

In [164]:
music = pd.read_csv('Merged.csv', header = 0, encoding = 'latin1')
genre_s = music['genre_top'].unique()
num_s = np.arange(1, genre_s.size + 1)
genre_map = pd.DataFrame([genre_s, num_s]).T
genre_map.columns = ['type', 'label']
music_label = music.merge(genre_map, left_on = 'genre_top', right_on = 'type', how = 'outer')

col_names = ['acousticness', 'danceability', 'energy', \
                       'instrumentalness', 'liveness', 'speechiness', \
                       'tempo', 'valence','label']

music_dnn = music_label[col_names]

training_data = music_dnn[music_dnn['label'] != 6].drop('label',axis = 1)
training_target = music_dnn[music_dnn['label'] != 6]['label']
targetting_data = music_dnn[music_dnn['label'] == 6].drop('label',axis = 1)
targetting_target = music_dnn[music_dnn['label'] == 6]['label']

def input_fn_train():
    x = tf.constant(training_data.as_matrix(), tf.float32, training_data.shape)
    y = tf.constant(training_target.as_matrix(), tf.int32, training_target.shape)
    return x, y

def input_fn_target():
    x = tf.constant(targetting_data.as_matrix(), tf.float32, targetting_data.shape)
    y = tf.constant(targetting_target.as_matrix(), tf.int32, targetting_target.shape)
    return x, y

feature_columns = [tf.contrib.layers.real_valued_column("", dimension=13)]

classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                              hidden_units=[10, 20, 10],
                                              n_classes=14 )

classifier.fit(input_fn=input_fn_train, steps=2000)

result = list(classifier.predict(input_fn=input_fn_target))

In [165]:
targetting_data['label'] = result
training_data = music_dnn[music_dnn['label'] != 6]
fin_frame = training_data.append(targetting_data)

In [166]:
fin_frame.shape

(13129, 9)

In [167]:
fin_frame.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,label
0,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,1
1,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,1
2,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,1
3,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,1
4,0.748986,0.765886,0.513173,0.957209,0.614844,0.886914,180.025,0.401672,1


In [168]:
label = fin_frame['label']
fin_frame_out = copy.copy(fin_frame)
del fin_frame_out['label']

In [169]:
col_names_temp = ['acousticness', 'danceability', 'energy', \
                       'instrumentalness', 'liveness', 'speechiness', \
                       'tempo', 'valence']

In [170]:
music_matrix = fin_frame_out.as_matrix()
scaler = StandardScaler()
scaler.fit(music_matrix)
new_matrix = scaler.transform(music_matrix)
music_std = pd.DataFrame(new_matrix, columns = col_names_temp)

In [171]:
music_std['label'] = label

In [172]:
music_std.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,label
0,-0.281499,0.991915,0.348732,-1.742887,-0.064267,0.437749,1.223573,0.495983,1
1,-0.391656,0.217484,1.006860,-1.767172,-0.518358,2.639805,0.110726,-0.617788,1
2,-1.253885,1.358338,0.589683,-1.770366,1.172705,0.185049,-0.651745,0.659017,1
3,-0.188870,0.136465,0.082343,-1.718498,-0.577286,3.103507,-0.251046,1.645950,1
4,0.584562,1.465209,-0.087551,0.876201,2.702027,5.734216,1.626357,-0.137994,1


In [173]:
genre_map.columns = ['genre_top', 'label']
fin_output_frame = pd.merge(music_std,genre_map)

In [174]:
fin_output_frame.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,label,genre_top
0,-0.281499,0.991915,0.348732,-1.742887,-0.064267,0.437749,1.223573,0.495983,1,Hip-Hop
1,-0.391656,0.217484,1.006860,-1.767172,-0.518358,2.639805,0.110726,-0.617788,1,Hip-Hop
2,-1.253885,1.358338,0.589683,-1.770366,1.172705,0.185049,-0.651745,0.659017,1,Hip-Hop
3,-0.188870,0.136465,0.082343,-1.718498,-0.577286,3.103507,-0.251046,1.645950,1,Hip-Hop
4,0.584562,1.465209,-0.087551,0.876201,2.702027,5.734216,1.626357,-0.137994,1,Hip-Hop


In [175]:
music_cat = pd.get_dummies(fin_output_frame['genre_top'])
nf = pd.concat([music_std,music_cat], axis = 1)

In [176]:
nf.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,label,Blues,...,Electronic,Experimental,Folk,Hip-Hop,Instrumental,International,Jazz,Old-Time / Historic,Pop,Rock
0,-0.281499,0.991915,0.348732,-1.742887,-0.064267,0.437749,1.223573,0.495983,1,0,...,0,0,0,1,0,0,0,0,0,0
1,-0.391656,0.217484,1.006860,-1.767172,-0.518358,2.639805,0.110726,-0.617788,1,0,...,0,0,0,1,0,0,0,0,0,0
2,-1.253885,1.358338,0.589683,-1.770366,1.172705,0.185049,-0.651745,0.659017,1,0,...,0,0,0,1,0,0,0,0,0,0
3,-0.188870,0.136465,0.082343,-1.718498,-0.577286,3.103507,-0.251046,1.645950,1,0,...,0,0,0,1,0,0,0,0,0,0
4,0.584562,1.465209,-0.087551,0.876201,2.702027,5.734216,1.626357,-0.137994,1,0,...,0,0,0,1,0,0,0,0,0,0


In [177]:
norm_data = copy.copy(nf)
del norm_data['label']

In [178]:
norm_data.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,Blues,Classical,Electronic,Experimental,Folk,Hip-Hop,Instrumental,International,Jazz,Old-Time / Historic,Pop,Rock
0,-0.281499,0.991915,0.348732,-1.742887,-0.064267,0.437749,1.223573,0.495983,0,0,0,0,0,1,0,0,0,0,0,0
1,-0.391656,0.217484,1.006860,-1.767172,-0.518358,2.639805,0.110726,-0.617788,0,0,0,0,0,1,0,0,0,0,0,0
2,-1.253885,1.358338,0.589683,-1.770366,1.172705,0.185049,-0.651745,0.659017,0,0,0,0,0,1,0,0,0,0,0,0
3,-0.188870,0.136465,0.082343,-1.718498,-0.577286,3.103507,-0.251046,1.645950,0,0,0,0,0,1,0,0,0,0,0,0
4,0.584562,1.465209,-0.087551,0.876201,2.702027,5.734216,1.626357,-0.137994,0,0,0,0,0,1,0,0,0,0,0,0


In [179]:
def input_fn():
  return tf.constant(norm_data.as_matrix(), tf.float32, norm_data.shape), None

In [180]:
tf.logging.set_verbosity(tf.logging.ERROR)
kmeans = tf.contrib.learn.KMeansClustering(num_clusters=9, 
relative_tolerance=0.0001)
kmeans.fit(input_fn=input_fn)

KMeansClustering(params={'num_clusters': 9, 'training_initial_clusters': 'random', 'distance_metric': 'squared_euclidean', 'random_seed': 0, 'use_mini_batch': True, 'mini_batch_steps_per_iteration': 1, 'kmeans_plus_plus_num_retries': 2, 'relative_tolerance': 0.0001})

In [181]:
clusters = kmeans.clusters()
print(clusters)
assignments = list(kmeans.predict_cluster_idx(input_fn=input_fn))

[[ -6.45863235e-01   1.04822516e+00   3.81450862e-01   5.15247166e-01
   -3.23317409e-01  -1.39639929e-01  -1.40028104e-01   6.77408099e-01
    1.87982991e-03   8.96763150e-03   2.44506851e-01   2.61390558e-03
    1.22511322e-02   7.72714242e-02   2.61886604e-03   1.49691999e-02
    4.03741840e-03   2.27067526e-02   2.64217760e-02   5.81755221e-01]
 [  8.10460687e-01   1.97940037e-01  -8.63045871e-01   6.50986075e-01
   -3.10042322e-01  -3.02861482e-01  -3.85353625e-01  -2.43014157e-01
    5.88858209e-04   1.48339272e-02   1.98478237e-01   5.22694318e-04
    1.63556978e-01   2.25155484e-02   1.04538864e-03   9.90472455e-03
    3.76273692e-02   1.22998552e-02   2.52216514e-02   5.13404846e-01]
 [  2.59994626e-01   5.54185569e-01  -1.74062088e-01  -8.43004346e-01
    8.05752128e-02   3.33280778e+00   4.16864380e-02   4.00719166e-01
    1.95671991e-03   3.36622214e-03   1.78177595e-01   0.00000000e+00
    3.12577747e-02   3.79354984e-01   1.65823767e-05   2.68634455e-03
    1.15081668e-02

In [182]:
kmeans.score(input_fn=input_fn, steps = 50)

60421.621

In [183]:
assign = pd.DataFrame(assignments)
output_frame = pd.concat([norm_data, assign], axis=1)
output_frame = output_frame.rename(columns={0: 'Cluster'})

In [184]:
output_frame.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,Blues,Classical,...,Experimental,Folk,Hip-Hop,Instrumental,International,Jazz,Old-Time / Historic,Pop,Rock,Cluster
0,-0.281499,0.991915,0.348732,-1.742887,-0.064267,0.437749,1.223573,0.495983,0,0,...,0,0,1,0,0,0,0,0,0,6
1,-0.391656,0.217484,1.006860,-1.767172,-0.518358,2.639805,0.110726,-0.617788,0,0,...,0,0,1,0,0,0,0,0,0,2
2,-1.253885,1.358338,0.589683,-1.770366,1.172705,0.185049,-0.651745,0.659017,0,0,...,0,0,1,0,0,0,0,0,0,6
3,-0.188870,0.136465,0.082343,-1.718498,-0.577286,3.103507,-0.251046,1.645950,0,0,...,0,0,1,0,0,0,0,0,0,2
4,0.584562,1.465209,-0.087551,0.876201,2.702027,5.734216,1.626357,-0.137994,0,0,...,0,0,1,0,0,0,0,0,0,2


In [185]:
music_fin_output = pd.read_csv('Merged.csv', header = 0, encoding = 'latin1')
columns = ['acousticness', 'danceability', 'energy', \
                       'instrumentalness', 'liveness', 'speechiness', \
                       'tempo', 'valence']
music_fin_output = music_fin_output[columns]
music_fin_output = pd.concat([music_fin_output,music_cat], axis = 1)
music_fin_output.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,Blues,Classical,Electronic,Experimental,Folk,Hip-Hop,Instrumental,International,Jazz,Old-Time / Historic,Pop,Rock
0,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,0,0,0,0,0,1,0,0,0,0,0,0
1,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,0,0,0,0,0,1,0,0,0,0,0,0
2,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,0,0,0,0,0,1,0,0,0,0,0,0
3,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,0,0,0,0,0,1,0,0,0,0,0,0
4,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,0,0,0,0,0,1,0,0,0,0,0,0


In [186]:
music_out_clusters = pd.concat([music_fin_output,output_frame], axis = 1)

In [187]:
track_id = pd.read_csv('Merged.csv', encoding='latin-1', usecols=['track_id'])
output_frame = pd.concat([track_id, music_out_clusters], axis=1)
output_frame.to_csv("Fin_Output_Cluster.csv")

In [188]:
silhouette_avg = silhouette_score(norm_data.as_matrix(), np.array(assignments))

In [189]:
silhouette_avg

0.12682559834761983

In [145]:
output_frame.head()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,label,...,Experimental,Folk,Hip-Hop,Instrumental,International,Jazz,Old-Time / Historic,Pop,Rock,Cluster
0,2,-0.281499,0.991915,0.348732,-1.742887,-0.064267,0.437749,1.223573,0.495983,1,...,0,0,1,0,0,0,0,0,0,0
1,3,-0.391656,0.217484,1.006860,-1.767172,-0.518358,2.639805,0.110726,-0.617788,1,...,0,0,1,0,0,0,0,0,0,2
2,5,-1.253885,1.358338,0.589683,-1.770366,1.172705,0.185049,-0.651745,0.659017,1,...,0,0,1,0,0,0,0,0,0,0
3,10,-0.188870,0.136465,0.082343,-1.718498,-0.577286,3.103507,-0.251046,1.645950,1,...,0,0,1,0,0,0,0,0,0,2
4,134,0.584562,1.465209,-0.087551,0.876201,2.702027,5.734216,1.626357,-0.137994,1,...,0,0,1,0,0,0,0,0,0,2


In [146]:
output_frame['track_id'] = output_frame['track_id'].astype(str)
output_frame['track_id'] = output_frame['track_id'].str.zfill(6)
fin_output_frame = output_frame[output_frame['track_id'].isin(filenames)]

# User Input

In [150]:
genre_theme = input("Enter Theme:")

Enter Theme:5


In [151]:
cluster_input=fin_output_frame[fin_output_frame['Cluster']==int(genre_theme)].track_id.tolist()

In [152]:
track = random.choice(cluster_input)

In [153]:
sep = '/'
tid_str = track
filename = AUDIO_DIR + sep + tid_str[:3] + sep + tid_str + '.mp3'
print (filename)

pygame.mixer.init() 
pygame.mixer.music.load(filename)
pygame.mixer.music.play()
pygame.mixer.music.fadeout(20000)

C:/Users/chapp/Courses/Python/Project/085/085419.mp3
